In [1]:
import os
import asyncio
from lightrag import LightRAG, QueryParam
from lightrag.llm import openai_complete_if_cache, openai_embedding
from lightrag.llm import ollama_model_complete, ollama_embedding
from lightrag.utils import EmbeddingFunc
from lightrag.kg.postgresql_impl import PostgresDB
import numpy as np
import textract
from dotenv import load_dotenv
import nest_asyncio 
nest_asyncio.apply() 

load_dotenv()

/home/luhg/anaconda3/envs/LightRAG/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
WORKING_DIR = "../data/test_postgres"

if not os.path.exists(WORKING_DIR):
    os.mkdir(WORKING_DIR)

In [3]:
async def llm_model_func(
    prompt, system_prompt=None, history_messages=[], **kwargs
) -> str:
    return await openai_complete_if_cache(
        "qwen-plus",
        prompt,
        system_prompt=system_prompt,
        history_messages=history_messages,
        api_key=os.getenv("DASHSCOPE_API_KEY"),
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
        **kwargs,
    )

In [4]:
postgresql_db = PostgresDB(
    config={
        "host": "localhost",
        "port": 6024,
        "database": "test_postgres",
        "user": "postgres",
        "password": "123456",
        "workspace": "test",
    }
)

In [5]:
print(postgresql_db.workspace)

test


In [6]:
rag = LightRAG(
    working_dir=WORKING_DIR,
    # kg="Neo4JStorage",
    llm_model_func=llm_model_func,
    embedding_func=EmbeddingFunc(
        embedding_dim=1024,
        max_token_size=512,
        func=lambda texts: ollama_embedding(
            texts, embed_model="viosay/conan-embedding-v1:latest", host="http://192.168.69.234:11343"
        ),
    ),
    chunk_token_size=500,
    log_level="DEBUG",
    vector_storage="PostgresVectorDBStorage"
)

rag.vector_db_storage_cls.db = postgresql_db

INFO:lightrag:Logger initialized for working directory: ../data/test_postgres
DEBUG:lightrag:LightRAG init with param:
  working_dir = ../data/test_postgres,
  kv_storage = JsonKVStorage,
  vector_storage = PostgresVectorDBStorage,
  graph_storage = NetworkXStorage,
  log_level = DEBUG,
  chunk_token_size = 500,
  chunk_overlap_token_size = 100,
  tiktoken_model_name = gpt-4o-mini,
  entity_extract_max_gleaning = 1,
  entity_summary_to_max_tokens = 500,
  node_embedding_algorithm = node2vec,
  node2vec_params = {'dimensions': 1536, 'num_walks': 10, 'walk_length': 40, 'window_size': 2, 'iterations': 3, 'random_seed': 3},
  embedding_func = {'embedding_dim': 1024, 'max_token_size': 512, 'func': <function <lambda> at 0x7f303eab72e0>},
  embedding_batch_num = 32,
  embedding_func_max_async = 16,
  llm_model_func = <function llm_model_func at 0x7f303eab7380>,
  llm_model_name = meta-llama/Llama-3.2-1B-Instruct,
  llm_model_max_token_size = 32768,
  llm_model_max_async = 16,
  llm_model_kwar

In [7]:
print(rag.vector_db_storage_cls.db.workspace)

test


In [8]:
from langchain_community.document_loaders.csv_loader import CSVLoader

doc_path = "../data/paper/scholat_paper_ed/scholat_paper_ed_001.csv"

loader = CSVLoader(doc_path)
data = loader.load()

data = [d.page_content for d in data]
need_to_insert_data = data[1:2]
need_to_insert_data

['title: 基于时限的角色访问控制委托模型\nauthors: 道炜 汤庸 冀高峰 杨虹轶\nsource: 计算机科学,2008,35（3）:175-177\nsourceDetail: \ndate: 2008.03\ntype: 期刊论文\nkeyword: \nsummary: ']

In [9]:
rag.entities_vdb.db = postgresql_db
rag.relationships_vdb.db = postgresql_db
rag.chunks_vdb.db = postgresql_db


await rag.entities_vdb.init_table()
await rag.relationships_vdb.init_table()
await rag.chunks_vdb.init_table()

INFO:lightrag:Connected to PostgreSQL at localhost:6024


In [10]:
await rag.ainsert(need_to_insert_data)

INFO:lightrag:[New Docs] inserting 1 docs
INFO:lightrag:[New Chunks] inserting 1 chunks
DEBUG:lightrag:[upserting chunks] {'chunk-64e1449282fc37680aa7d6f2e4a9d92a': {'tokens': 76, 'content': 'title: 基于时限的角色访问控制委托模型\nauthors: 道炜 汤庸 冀高峰 杨虹轶\nsource: 计算机科学,2008,35（3）:175-177\nsourceDetail: \ndate: 2008.03\ntype: 期刊论文\nkeyword: \nsummary:', 'chunk_order_index': 0, 'full_doc_id': 'doc-64e1449282fc37680aa7d6f2e4a9d92a'}}
INFO:httpx:HTTP Request: POST http://192.168.69.234:11343/api/embed "HTTP/1.1 200 OK"
ERROR:lightrag:PostgreSQL error: invalid input for query argument $4: 76 (expected str, got int)
Query: 
        INSERT INTO light_chunks (id,embedding,workspace,tokens,content,full_doc_id,chunk_order_index)
        VALUES ($1,$2,$3,$4,$5,$6,$7)
        ON CONFLICT (id) DO UPDATE SET
        embedding=excluded.embedding,workspace=excluded.workspace,tokens=excluded.tokens,content=excluded.content,full_doc_id=excluded.full_doc_id,chunk_order_index=excluded.chunk_order_index;
        
ERROR:li

### 可视化本地图为网页

In [ ]:
import networkx as nx
from pyvis.network import Network
import random

# Load the GraphML file
G = nx.read_graphml("../data/test_paper/graph_chunk_entity_relation.graphml")

# Create a Pyvis network
net = Network(height="100vh", notebook=True)

# Convert NetworkX graph to Pyvis network
net.from_nx(G)

# Add colors to nodes
for node in net.nodes:
    node["color"] = "#{:06x}".format(random.randint(0, 0xFFFFFF))

# Save and display the network
net.show("../data/test_paper/knowledge_graph.html")


### 可视化本地图到 neo4j

In [ ]:
import os
import json
from lightrag.utils import xml_to_json
from neo4j import GraphDatabase

# Constants
BATCH_SIZE_NODES = 500
BATCH_SIZE_EDGES = 100

# Neo4j connection credentials
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "12345678"


def convert_xml_to_json(xml_path, output_path):
    """Converts XML file to JSON and saves the output."""
    if not os.path.exists(xml_path):
        print(f"Error: File not found - {xml_path}")
        return None

    json_data = xml_to_json(xml_path)
    if json_data:
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(json_data, f, ensure_ascii=False, indent=2)
        print(f"JSON file created: {output_path}")
        return json_data
    else:
        print("Failed to create JSON data")
        return None


def process_in_batches(tx, query, data, batch_size):
    """Process data in batches and execute the given query."""
    for i in range(0, len(data), batch_size):
        batch = data[i : i + batch_size]
        tx.run(query, {"nodes": batch} if "nodes" in query else {"edges": batch})


def main():
    # Paths
    xml_file = os.path.join(WORKING_DIR, "graph_chunk_entity_relation.graphml")
    json_file = os.path.join(WORKING_DIR, "graph_data.json")

    # Convert XML to JSON
    json_data = convert_xml_to_json(xml_file, json_file)
    if json_data is None:
        return

    # Load nodes and edges
    nodes = json_data.get("nodes", [])
    edges = json_data.get("edges", [])

    # Neo4j queries
    create_nodes_query = """
    UNWIND $nodes AS node
    MERGE (e:Entity {id: node.id})
    SET e.entity_type = node.entity_type,
        e.description = node.description,
        e.source_id = node.source_id,
        e.displayName = node.id
    REMOVE e:Entity
    WITH e, node
    CALL apoc.create.addLabels(e, [node.entity_type]) YIELD node AS labeledNode
    RETURN count(*)
    """

    create_edges_query = """
    UNWIND $edges AS edge
    MATCH (source {id: edge.source})
    MATCH (target {id: edge.target})
    WITH source, target, edge,
         CASE
            WHEN edge.keywords CONTAINS 'lead' THEN 'lead'
            WHEN edge.keywords CONTAINS 'participate' THEN 'participate'
            WHEN edge.keywords CONTAINS 'uses' THEN 'uses'
            WHEN edge.keywords CONTAINS 'located' THEN 'located'
            WHEN edge.keywords CONTAINS 'occurs' THEN 'occurs'
           ELSE REPLACE(SPLIT(edge.keywords, ',')[0], '\"', '')
         END AS relType
    CALL apoc.create.relationship(source, relType, {
      weight: edge.weight,
      description: edge.description,
      keywords: edge.keywords,
      source_id: edge.source_id
    }, target) YIELD rel
    RETURN count(*)
    """

    set_displayname_and_labels_query = """
    MATCH (n)
    SET n.displayName = n.id
    WITH n
    CALL apoc.create.setLabels(n, [n.entity_type]) YIELD node
    RETURN count(*)
    """

    # Create a Neo4j driver
    driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

    try:
        # Execute queries in batches
        with driver.session() as session:
            # Insert nodes in batches
            session.execute_write(
                process_in_batches, create_nodes_query, nodes, BATCH_SIZE_NODES
            )

            # Insert edges in batches
            session.execute_write(
                process_in_batches, create_edges_query, edges, BATCH_SIZE_EDGES
            )

            # Set displayName and labels
            session.run(set_displayname_and_labels_query)

    except Exception as e:
        print(f"Error occurred: {e}")

    finally:
        driver.close()


main()


In [ ]:
print(
    await rag.aquery(
        "汤庸发表过哪些论文？",
        param=QueryParam(mode="hybrid"),
    )
)